In [1]:
import pandas as pd

train = pd.read_csv('..\\0.data\\raw\\imdb_train.csv')
print(len(train))
print(train.head())

17500
      id  labels                                               text
0   1288       0  We saw this on the shelf at the local video st...
1   2064       0  Well, you'd better if you plan on sitting thro...
2  18997       1  This is my favorite Jackie Chan movie and in a...
3  10448       0  The long list of "big" names in this flick (in...
4  16133       1  The great and underrated Marion Davies shows h...


In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk import stem

def clean_paragraph(para):
    lmtzr = stem.WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+')
    
    p = ' '.join([lmtzr.lemmatize(token.lower()) for token in tokenizer.tokenize(para)])
    
    return p

In [3]:
import time; t0 = time.time()

train.text = [clean_paragraph(para) for para in train.text]

print(time.time() - t0)

43.23121356964111


In [4]:
train.text[0]

'we saw this on the shelf at the local video store saw coppola in the credit and got excited that wa the one and only time this movie raised any interest i could never quite work out if it wa an attempt at a humourous film that failed miserably or an attempt at a serious film that failed miserably in general the entire production seemed incredibly amatuerish the sound in particular wa absolutely dreadful especially in the scene shot in the little bar the dialogue wa so corny in part it wa unbelievable very disappointing'

## Create Classes and Assign Probabilities

In [5]:
'''Find probabilites of each class'''

print(train.labels.value_counts())

prob_of_each_class = []
totalDocCount = len(train)

for a in train.labels.value_counts():
    prob_of_each_class.append(float(a/totalDocCount))
    
prob_of_each_class = pd.Series(prob_of_each_class)
print(prob_of_each_class); prob_of_each_class.sum()

0    8764
1    8736
Name: labels, dtype: int64
0    0.5008
1    0.4992
dtype: float64


1.0

## Create Word Matrix

In [6]:
'''Get Unique Words from the text data'''

tokenDict = {}
i = 0 
for row in train.text:
    for token in row.split(' '):
        if tokenDict.get(token) == None:
            tokenDict[token] = i
            i = i + 1
        else:
            continue

len(tokenDict)

58535

## Initialize Matrix

In [7]:
from scipy.sparse import coo_matrix
import numpy as np

matrix = coo_matrix((3, 58535))
matrix = matrix.tocsr()
matrix

<3x58535 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

## Fill Matrix 

In [ ]:
'''Fill the Matrix'''

for index, row in train.iterrows():
    for token in row['text'].split(' '):
        matrix[row['labels'], tokenDict[token]] = matrix[row['labels'], tokenDict[token]] + 1
        matrix[2, tokenDict[token]] = matrix[2, tokenDict[token]] + 1

C:\Users\rritesh\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\sparse\compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [ ]:
'''Verify Matrix'''

print(matrix[:, 0].sum())

In [ ]:
print(matrix[:, 0])
print(wordCountVector[tokenDict['and']])

## Make Predictions on Test Data - Load, Clean, Predict Prob

In [ ]:
'''Load Test Data'''
test = pd.read_csv('..\\0.data\\raw\\imdb_test.csv')
test['Pred_Labels'] = None

print(len(test))
print(test.head())

In [ ]:
'''Clean Test Commentary Text Data'''

import time; t0 = time.time()

test.text = [clean_paragraph(para) for para in test.text]

print(time.time() - t0)

In [ ]:
for index, row in test.iterrows():
    pred_class_prob = 0.0
    pred_class = 0
    
    for target_class in range(0, 2):
        product = prob_of_each_class[target_class]
        
        for token in row['text'].split(' '):
            if tokenDict.get(token) == None:
                continue
                
            p_value = matrix[target_class, tokenDict[token]] / matrix[2, tokenDict[token]]
            
            if p_value != 0.0:
                product = product * p_value
                
        if product > pred_class_prob:
            pred_class = target_class
            pred_class_prob = product
            
    test.at[index, 'Pred_Labels'] = pred_class

## Find Accuracy Score

In [ ]:
correct_count = 0
total = 0

for index, row in test.iterrows():
    if str(row['Pred_labels']) = str(row['labels']):
        correct_count = correct_count + 1
    
    total = total + 1
    
print('Accuracy: ', correct_count/total)